# Proyecto 1 – Limpieza de Establecimientos Educativos (Diversificado)

Este cuaderno realiza **exclusivamente limpieza** (sin análisis) sobre el dataset unificado `centros_educativos_completo.csv`. Incluye bitácora de acciones y marcadores de posibles duplicados.

## Contenido
1. Carga de datos crudos
2. Utilidades de normalización
3. Limpieza general (espacios/guiones/vacíos)
4. Limpieza específica de `ESTABLECIMIENTO`, `DIRECCION`, `TELEFONO`
5. Estandarización de categóricas
6. Generación de clave canónica y marcaje de posibles duplicados
7. Guardado de salidas (CSV limpio, duplicados, bitácora)
8. (Opcional) Búsqueda de un centro educativo


In [ ]:
import re, unicodedata
import pandas as pd
from collections import OrderedDict
from datetime import datetime

csv_in = '/mnt/data/centros_educativos_completo.csv'
df = pd.read_csv(csv_in, dtype=str)
print('Filas x columnas:', df.shape)
df.head(3)

## Utilidades de normalización

In [ ]:
def strip_accents(s: str) -> str:
    if not isinstance(s, str):
        return s
    return ''.join(ch for ch in unicodedata.normalize('NFKD', s) if not unicodedata.combining(ch))

def collapse_spaces_and_hyphens(s: str) -> str:
    if not isinstance(s, str):
        return s
    s = s.replace('—', '-').replace('–', '-').replace('−', '-').replace('­', '')
    s = re.sub(r'[ \t\u00A0]+', ' ', s)
    s = re.sub(r'-{2,}', '-', s)
    s = re.sub(r'\s*-\s*', ' - ', s)
    s = re.sub(r'\s{2,}', ' ', s)
    return s.strip()

def normalize_text_basic(s: str) -> str:
    if not isinstance(s, str):
        return s
    s = s.strip().replace('"', '').replace('“','').replace('”','').replace('´', "'")
    s = strip_accents(s).upper()
    s = collapse_spaces_and_hyphens(s)
    return s

def expand_address_abbrev(s: str) -> str:
    if not isinstance(s, str):
        return s
    txt = ' ' + s + ' '
    rules = OrderedDict([
        (r'\bAV[\.]?\b', ' AVENIDA '),
        (r'\bAVE[\.]?\b', ' AVENIDA '),
        (r'\bAVDA[\.]?\b', ' AVENIDA '),
        (r'\bBLVD[\.]?\b', ' BOULEVARD '),
        (r'\bCALZ[\.]?\b', ' CALZADA '),
        (r'\bCOL[\.]?\b', ' COLONIA '),
        (r'\bCOND[\.]?\b', ' CONDOMINIO '),
        (r'\bRES[\.]?\b', ' RESIDENCIAL '),
        (r'\bZ[\.]?\b(?=\s*\d)', ' ZONA '),
        (r'\bZONA\b\s*(?=\d)', ' ZONA '),
        (r'\bKM[\.]?\b', ' KM '),
        (r'\bNO[\.]?\b', ' NUMERO '),
        (r'\bN[\.]?\b(?=\s*\d)', ' NUMERO '),
        (r'\b#\b', ' NUMERO '),
        (r'\bEDIF[\.]?\b', ' EDIFICIO '),
        (r'\b2DA\b', ' 2A '),
        (r'\b3RA\b', ' 3A '),
    ])
    for pat, rep in rules.items():
        txt = re.sub(pat, rep, txt, flags=re.IGNORECASE)
    txt = collapse_spaces_and_hyphens(txt)
    return txt

def normalize_name(s: str) -> str:
    return normalize_text_basic(s)

def normalize_address(s: str) -> str:
    if not isinstance(s, str):
        return s
    s = normalize_text_basic(s)
    s = expand_address_abbrev(s)
    s = re.sub(r'\b(\d+)\s*\.\s*', r'\1 ', s)
    s = re.sub(r'\bKM\.\b', ' KM ', s)
    s = collapse_spaces_and_hyphens(s)
    return s

def clean_phone_field(s: str):
    if not isinstance(s, str):
        return None
    nums = re.findall(r'\d{8}', s)
    nums = [n for n in nums if not re.fullmatch(r'0{8}', n)]
    seen, unique = set(), []
    for n in nums:
        if n not in seen:
            seen.add(n)
            unique.append(n)
    return ' / '.join(unique) if unique else None

def make_duplicate_key(nombre, direccion, municipio, departamento) -> str:
    nombre_n = normalize_name(nombre) if isinstance(nombre, str) else ''
    direccion_n = normalize_address(direccion) if isinstance(direccion, str) else ''
    municipio_n = normalize_text_basic(municipio) if isinstance(municipio, str) else ''
    depto_n = normalize_text_basic(departamento) if isinstance(departamento, str) else ''
    key = f"{nombre_n} | {direccion_n} | {municipio_n} | {depto_n}"
    key = collapse_spaces_and_hyphens(key)
    return key


## Bitácora de acciones

In [ ]:
bitacora = []
def log_step(accion: str, justificacion: str, resumen=None):
    bitacora.append({
        'timestamp': datetime.now().isoformat(timespec='seconds'),
        'accion': accion,
        'justificacion': justificacion,
        'resumen': resumen or {}
    })
log_step('cargar_datos', 'Cargar CSV crudo como texto para preservar formatos originales.',
         {'filas': len(df), 'columnas': list(df.columns)})
len(bitacora)

## Limpieza general (espacios/guiones/vacíos)

In [ ]:
object_cols = [c for c in df.columns if df[c].dtype == 'object']
na_antes = {c: int(df[c].isna().sum()) for c in object_cols}

def _collapse_series(s: pd.Series) -> pd.Series:
    return s.map(lambda x: collapse_spaces_and_hyphens(x) if isinstance(x, str) else x)

df[object_cols] = df[object_cols].apply(_collapse_series)
df[object_cols] = df[object_cols].replace({'': pd.NA, 'nan': pd.NA, 'None': pd.NA})
na_despues = {c: int(df[c].isna().sum()) for c in object_cols}
log_step('normalizar_espacios_guiones', 'Colapsar espacios múltiples/guiones y estandarizar vacíos como NA.',
         {'na_antes': na_antes, 'na_despues': na_despues})
len(bitacora)

## Limpieza específica de ESTABLECIMIENTO, DIRECCION y TELEFONO

In [ ]:
for col in ['ESTABLECIMIENTO','DIRECCION','TELEFONO']:
    if col in df.columns and f'{col}_ORIG' not in df.columns:
        df[f'{col}_ORIG'] = df[col]

placeholders = re.compile(r'^[-–—]{1,}$')
cols_sensitive = ['ESTABLECIMIENTO','DIRECCION','TELEFONO','MUNICIPIO','DEPARTAMENTO']
replaced = {}
for col in cols_sensitive:
    if col in df.columns:
        count_before = int(df[col].notna().sum())
        df.loc[df[col].astype(str).str.fullmatch(placeholders, na=False), col] = pd.NA
        replaced[col] = {'no_nulos_antes': count_before, 'no_nulos_despues': int(df[col].notna().sum())}
log_step('placeholder_a_na', 'Reemplazar campos compuestos solo por guiones por NA.', replaced)

before_tel = int(df['TELEFONO'].notna().sum()) if 'TELEFONO' in df.columns else None
if 'ESTABLECIMIENTO' in df.columns:
    df['ESTABLECIMIENTO'] = df['ESTABLECIMIENTO'].map(lambda x: normalize_name(x) if isinstance(x, str) else x)
if 'DIRECCION' in df.columns:
    df['DIRECCION'] = df['DIRECCION'].map(lambda x: normalize_address(x) if isinstance(x, str) else x)
if 'TELEFONO' in df.columns:
    df['TELEFONO'] = df['TELEFONO'].map(lambda x: clean_phone_field(x) if isinstance(x, str) else x)
    df['TELEFONO_VALIDO'] = df['TELEFONO'].apply(lambda x: bool(x) if pd.notna(x) else False)
after_tel_valid = int(df['TELEFONO_VALIDO'].sum()) if 'TELEFONO' in df.columns else None
log_step('normalizar_nombre_direccion_telefono', 'Mayúsculas sin acentos, abreviaturas en dirección y extracción de teléfonos de 8 dígitos.',
         {'telefonos_no_na_antes': before_tel, 'telefonos_validos_despues': after_tel_valid})
df[['ESTABLECIMIENTO','DIRECCION','TELEFONO']].head(3)

## Estandarización de categóricas

In [ ]:
for col in ['DEPARTAMENTO','MUNICIPIO','SECTOR','AREA','STATUS','MODALIDAD','JORNADA','PLAN','NIVEL']:
    if col in df.columns:
        df[col] = df[col].map(lambda x: normalize_text_basic(x) if isinstance(x, str) else x)
log_step('estandarizar_categoricas', 'Estandarizar en mayúsculas sin acentos para evitar variantes por caso/acentos.')
sorted(df['JORNADA'].dropna().unique().tolist()) if 'JORNADA' in df.columns else 'OK'

## Clave canónica y marcaje de posibles duplicados

In [ ]:
for required in ['ESTABLECIMIENTO','DIRECCION','MUNICIPIO','DEPARTAMENTO']:
    if required not in df.columns:
        df[required] = pd.NA
df['CLAVE_DUP'] = df.apply(lambda r: make_duplicate_key(r['ESTABLECIMIENTO'], r['DIRECCION'], r['MUNICIPIO'], r['DEPARTAMENTO']), axis=1)
grp_sizes = df.groupby('CLAVE_DUP')['CLAVE_DUP'].transform('size')
df['POSIBLE_DUPLICADO'] = grp_sizes.gt(1)
df_dups = df.loc[df['POSIBLE_DUPLICADO'], ['CLAVE_DUP','CODIGO','ESTABLECIMIENTO','DIRECCION','TELEFONO','MUNICIPIO','DEPARTAMENTO','JORNADA','PLAN']] \
            .sort_values(['DEPARTAMENTO','MUNICIPIO','ESTABLECIMIENTO','DIRECCION'])
log_step('marcar_posibles_duplicados', 'Agrupar registros que coinciden en nombre+dirección+ubicación para revisión.',
         {'clusters_sospechosos': int(df_dups['CLAVE_DUP'].nunique()), 'filas_en_clusters': int(len(df_dups))})
df_dups.head(10)

## Guardado de salidas

In [ ]:
csv_out_clean = '/mnt/data/centros_educativos_completo_limpio.csv'
csv_out_dups  = '/mnt/data/posibles_duplicados.csv'
csv_out_log   = '/mnt/data/bitacora_limpieza.csv'
df.to_csv(csv_out_clean, index=False, encoding='utf-8')
df_dups.to_csv(csv_out_dups, index=False, encoding='utf-8')
pd.DataFrame(bitacora).to_csv(csv_out_log, index=False, encoding='utf-8')
print('Guardados:')
print(csv_out_clean)
print(csv_out_dups)
print(csv_out_log)

## (Opcional) Función de búsqueda de un centro educativo

In [ ]:
def buscar_centro(df, nombre=None, codigo=None, municipio=None, departamento=None):
    q = pd.Series([True]*len(df))
    if nombre:
        q &= df['ESTABLECIMIENTO'].str.contains(nombre, case=False, na=False)
    if codigo:
        q &= (df['CODIGO'] == codigo)
    if municipio:
        q &= df['MUNICIPIO'].str.contains(municipio, case=False, na=False)
    if departamento:
        q &= df['DEPARTAMENTO'].str.contains(departamento, case=False, na=False)
    cols = ['CODIGO','ESTABLECIMIENTO','DIRECCION','TELEFONO','MUNICIPIO','DEPARTAMENTO','JORNADA','PLAN']
    cols = [c for c in cols if c in df.columns]
    return df.loc[q, cols].sort_values(cols)

# Ejemplo:
# buscar_centro(df, nombre='INMACULADA', municipio='GUATEMALA')